In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import pandas_gbq
import copy
import copy
import pickle
import datetime as dt
from datetime import timezone
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
from google.cloud import bigquery
client = bigquery.Client('turing-230020')
def GBQ_data(query_string):
    query = client.query(query_string)
    results = query.result()
    return results.to_dataframe()

In [4]:
# Data set for analysisng the ACC cheating Devs
acc_query = """
WITH acc_data AS (
select 
  challenge_id,
  user_id,
  total_score_by_cases,
  total_cases,
  total_time,
  ROW_NUMBER()OVER(PARTITION BY user_id, challenge_id ORDER BY submit_time DESC) AS row_order
from 
  devdb_mirror.dv2_challenge_submit
WHERE
  submit_time BETWEEN '2022-07-01' AND '2022-07-31'
  AND challenge_id IN (220,218)
  AND bypass = 0
  AND passed = 1
  --AND total_time >= 0
)
SELECT
  challenge_id,
  user_id,
  total_score_by_cases,
  total_cases,
  total_time,
  100.0 * total_score_by_cases/ total_cases AS pct_score
FROM 
  acc_data
WHERE
  row_order = 1
"""
acc_dataframe = GBQ_data(acc_query)
acc_dataframe

,challenge_id,user_id,total_score_by_cases,total_cases,total_time,pct_score
0,220,84582,9,12,1028,75.000000
1,218,118299,3,12,951,25.000000
2,218,126572,9,12,16296,75.000000
3,220,431639,6,12,1421,50.000000
4,220,473314,6,12,1802,50.000000
...,...,...,...,...,...,...
7723,220,2533749,9,12,1301,75.000000
7724,218,2534405,12,12,1893,100.000000
7725,218,2537344,10,12,1192,83.333333
7726,218,2537968,12,12,1539,100.000000


In [5]:
## Calculating quartile value for pct_score column for ACC
Q3_acc_score = acc_dataframe['pct_score'].quantile(0.75)
Q1_acc_score = acc_dataframe['pct_score'].quantile(0.25)
Q1_acc_score, Q3_acc_score

(50.0, 100.0)

In [6]:
# For outlier ACC scores, we need to calculate only upper outlier as high scoring devs might be cheating
upper_acc_outlier_score = Q3_acc_score + 1.5 * (Q3_acc_score - Q1_acc_score)
upper_acc_outlier_score

175.0

In [7]:
# Since we are taking percentage value as score we cannot consider the outlier using IQR method. We should take the Q4 value for ACC scores.

In [9]:
# Calculating quartile value for total_time column from ACC dataset
Q3_acc_time = acc_dataframe['total_time'].quantile(0.75)
Q1_acc_time = acc_dataframe['total_time'].quantile(0.25)
Q1_acc_time, Q3_acc_time

(938.75, 1757)

In [10]:
# For ACC outlier time, we need to calculate only lower outlier as low time taking devs might be cheating

lower_acc_outlier_time = Q1_acc_time - 1.5* (Q3_acc_time - Q1_acc_time)
lower_acc_outlier_time

-288.625

In [11]:
# Variables to define cheating threshold 

score_cheat_acc = Q3_acc_score
time_cheat_acc = Q1_acc_time
score_cheat_acc, time_cheat_acc

(100.0, 938.75)

In [13]:
# Adding a column to determine if a dev is cheat
acc_dataframe['outlier_devs'] = np.where((acc_dataframe['pct_score'] > score_cheat_acc) & (acc_dataframe['total_time'] < time_cheat_acc),1,0)
acc_dataframe

,challenge_id,user_id,total_score_by_cases,total_cases,total_time,pct_score,outlier_devs
0,220,84582,9,12,1028,75.000000,0
1,218,118299,3,12,951,25.000000,0
2,218,126572,9,12,16296,75.000000,0
3,220,431639,6,12,1421,50.000000,0
4,220,473314,6,12,1802,50.000000,0
...,...,...,...,...,...,...,...
7723,220,2533749,9,12,1301,75.000000,0
7724,218,2534405,12,12,1893,100.000000,0
7725,218,2537344,10,12,1192,83.333333,0
7726,218,2537968,12,12,1539,100.000000,0


In [14]:
# creating dataframe for potentially cheat devs 
acc_outlier_dev = acc_dataframe[acc_dataframe['outlier_devs'] == 1].reset_index(drop=True)
acc_outlier_dev

,challenge_id,user_id,total_score_by_cases,total_cases,total_time,pct_score,outlier_devs
0,220,2526991,13,12,643,108.333333,1
1,220,122180,13,12,929,108.333333,1
2,220,2486297,13,12,685,108.333333,1
3,220,2485451,13,12,704,108.333333,1
4,220,2363875,13,12,738,108.333333,1
5,220,2537632,13,12,733,108.333333,1
6,220,2501515,13,12,664,108.333333,1
7,220,2497406,13,12,705,108.333333,1
8,220,1965231,13,12,590,108.333333,1
9,220,2206632,13,12,876,108.333333,1
